In [35]:
!pip install pandas

!pip install requests

     |████████████████████████████████| 61 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 153 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 16.1 MB/s eta 0:00:01


In [37]:
import pandas as pd
import gmaps
import requests 

In [14]:
from configgmaps import g_key

In [15]:

city_data_df = pd.read_csv('Weather_database.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,0,san ignacio,17.1561,-89.0714,78.80,scattered clouds,0.0,0.0,69,40,6.91,BZ
1,1,marienburg,54.0359,19.0266,36.00,few clouds,0.0,0.0,100,16,6.35,PL
2,2,cape town,-33.9258,18.4232,63.00,clear sky,0.0,0.0,82,0,6.91,ZA
3,3,rikitea,-23.1203,-134.9692,71.44,heavy intensity rain,4.4,0.0,91,100,3.91,PF
4,4,deputatskiy,69.3000,139.9000,6.67,overcast clouds,0.0,0.0,88,100,2.84,RU


In [16]:
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?60
What is the maximum temperature you would like for your trip?200


In [25]:
filtered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                          (city_data_df['Max Temp'] >= min_temp)]


In [26]:
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,0,san ignacio,17.1561,-89.0714,78.80,scattered clouds,0.0,0.0,69,40,6.91,BZ
2,2,cape town,-33.9258,18.4232,63.00,clear sky,0.0,0.0,82,0,6.91,ZA
3,3,rikitea,-23.1203,-134.9692,71.44,heavy intensity rain,4.4,0.0,91,100,3.91,PF
6,6,kapaa,22.0752,-159.3190,78.80,broken clouds,0.0,0.0,65,75,18.41,US
7,7,saint-philippe,-21.3585,55.7679,77.00,overcast clouds,0.0,0.0,73,90,10.36,RE


In [27]:
filtered_cities_df.count()

City_ID                295
City                   295
Lat                    295
Lng                    295
Max Temp               295
Current Description    295
Rain (inches)          295
Snow (inches)          295
Humidity               295
Cloudiness             295
Wind Speed             295
Country                292
dtype: int64

In [28]:
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,0,san ignacio,17.1561,-89.0714,78.80,scattered clouds,0.000,0.0,69,40,6.91,BZ
2,2,cape town,-33.9258,18.4232,63.00,clear sky,0.000,0.0,82,0,6.91,ZA
3,3,rikitea,-23.1203,-134.9692,71.44,heavy intensity rain,4.400,0.0,91,100,3.91,PF
6,6,kapaa,22.0752,-159.3190,78.80,broken clouds,0.000,0.0,65,75,18.41,US
7,7,saint-philippe,-21.3585,55.7679,77.00,overcast clouds,0.000,0.0,73,90,10.36,RE
...,...,...,...,...,...,...,...,...,...,...,...,...
550,550,padang,-0.9492,100.3543,86.00,scattered clouds,0.000,0.0,70,40,6.91,ID
554,554,banda aceh,5.5577,95.3222,82.42,moderate rain,1.153,0.0,70,100,18.90,ID
556,556,nicolas bravo,18.4500,-88.9167,82.00,broken clouds,0.000,0.0,79,78,1.99,MX
560,560,ikongo,-21.8833,47.4333,68.07,light rain,0.340,0.0,98,99,0.34,MG


In [30]:
hotel_df = filtered_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng', 'Current Description']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,san ignacio,BZ,78.80,17.1561,-89.0714,scattered clouds,
2,cape town,ZA,63.00,-33.9258,18.4232,clear sky,
3,rikitea,PF,71.44,-23.1203,-134.9692,heavy intensity rain,
6,kapaa,US,78.80,22.0752,-159.3190,broken clouds,
7,saint-philippe,RE,77.00,-21.3585,55.7679,overcast clouds,


In [46]:
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

# Iterate through.
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except IndexError:
        print('Hotel not found... skipping...')
   

Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...
Hotel not found... skipping...


In [47]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,san ignacio,BZ,78.80,17.1561,-89.0714,scattered clouds,Rainforest Haven Inn
2,cape town,ZA,63.00,-33.9258,18.4232,clear sky,Southern Sun Waterfront Cape Town
3,rikitea,PF,71.44,-23.1203,-134.9692,heavy intensity rain,Pension Maro'i
6,kapaa,US,78.80,22.0752,-159.3190,broken clouds,Sheraton Kauai Resort at Coconut Beach
7,saint-philippe,RE,77.00,-21.3585,55.7679,overcast clouds,"Chambres d'hôte ""La Trinité"""


In [50]:
filtered_hotels_df = hotel_df.dropna()
filtered_hotels_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,san ignacio,BZ,78.80,17.1561,-89.0714,scattered clouds,Rainforest Haven Inn
2,cape town,ZA,63.00,-33.9258,18.4232,clear sky,Southern Sun Waterfront Cape Town
3,rikitea,PF,71.44,-23.1203,-134.9692,heavy intensity rain,Pension Maro'i
6,kapaa,US,78.80,22.0752,-159.3190,broken clouds,Sheraton Kauai Resort at Coconut Beach
7,saint-philippe,RE,77.00,-21.3585,55.7679,overcast clouds,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
550,padang,ID,86.00,-0.9492,100.3543,scattered clouds,Mercure Padang
554,banda aceh,ID,82.42,5.5577,95.3222,moderate rain,Sulthan Hotel International
556,nicolas bravo,MX,82.00,18.4500,-88.9167,broken clouds,
560,ikongo,MG,68.07,-21.8833,47.4333,light rain,


In [51]:
output_data_file = './Weather_Vacation.csv'
hotel_df.to_csv(output_data_file, index_label='City_ID')

In [55]:
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} at {Max Temp}</dd>

"""

hotel_df = pd.read_csv('./Weather_Vacation.csv')
hotel_df.head()


,City_ID,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,0,san ignacio,BZ,78.80,17.1561,-89.0714,scattered clouds,Rainforest Haven Inn
1,2,cape town,ZA,63.00,-33.9258,18.4232,clear sky,Southern Sun Waterfront Cape Town
2,3,rikitea,PF,71.44,-23.1203,-134.9692,heavy intensity rain,Pension Maro'i
3,6,kapaa,US,78.80,22.0752,-159.3190,broken clouds,Sheraton Kauai Resort at Coconut Beach
4,7,saint-philippe,RE,77.00,-21.3585,55.7679,overcast clouds,"Chambres d'hôte ""La Trinité"""


In [57]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[['Lat', 'Lng']]

In [60]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))